In [17]:
import math
import torch.nn as nn

class Crack(nn.Module):
    def __init__(self, Crack_cfg):
        super(Crack, self).__init__()
        self.features = self._make_layers(Crack_cfg)
        # linear layer
#         self.classifier = nn.Linear(512, 10)
#         self.linear1 = nn.Linear(32*6*6,64)
#         self.linear2 = nn.Linear(64,64)
#         self.linear3 = nn.Linear(64,25)
        self.classifier = self.make_classifier()
    
    def make_classifier(self):
        classifier = []
        classifier += [nn.Linear(32*6*6,64),nn.ReLU(inplace=True),nn.Dropout(p=0.5)]
        classifier += [nn.Linear(64,64),nn.ReLU(inplace=True),nn.Dropout(p=0.5)]
        classifier += [nn.Linear(64,25)]
        return nn.Sequential(*classifier)
    
    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
#         out = self.linear1(out)
#         out = self.linear2(out)
#         out = self.linear3(out)
        return out

    def _make_layers(self, cfg):
        """
        cfg: a list define layers this layer contains
            'M': MaxPool, number: Conv2d(out_channels=number) -> BN -> ReLU
        """
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
            
#         layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)

In [18]:
Crack_cfg = {
    'Crack11':[16,16,'M',32,32,'M']
}
cracknet = Crack(Crack_cfg['Crack11']);
print(cracknet)

Crack(
  (features): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU(inplace)
    (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
    (5): ReLU(inplace)
    (6): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
    (7): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
    (9): ReLU(inplace)
    (10): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
    (12): ReLU(inplace)
    (13): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=1152, out_features=64, bias=True)
    (1): ReLU(inplace)
    (2): Dropout(p=0.5)
   

In [3]:
def train(model,trainloader,testloader,device,lr,optimizer,epoches):
    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = optimizer;
    running_loss = []
    
    running_accuracy = []
    
    temp_loss = 0.0
    
    # count the iteration number in an epoch
    iteration = 0 
    
    for epoch in range(epoches):
        for i,data in enumerate(trainloader):
            inputs,label = data
            inputs,label = inputs.to(device),label.to(device)
            
            outputs = model(inputs)
            
            optimizer.zero_grad()
            
            loss = criterion(outputs,label)
            
            loss.backward()
            
            optimizer.step()
            
            temp_loss += loss.item()
            iteration += 1

In [4]:
def fit(model, num_epochs, optimizer, device):
    """
    train and evaluate an classifier num_epochs times.
    We use optimizer and cross entropy loss to train the model. 
    Args: 
        model: CNN network
        num_epochs: the number of training epochs
        optimizer: optimize the loss function
    """
        
    # loss and optimizer
    loss_func = nn.CrossEntropyLoss()
    model.to(device)
    loss_func.to(device)
    
    # log train loss and test accuracy
    losses = []
    accs = []
    
    for epoch in range(num_epochs):
        
        print('Epoch {}/{}:'.format(epoch + 1, num_epochs))
        # train step
        loss = train(model, train_loader, loss_func, optimizer, device)
        losses.append(loss)
        
        # evaluate step
        accuracy = evaluate(model, test_loader, device)
        accs.append(accuracy)
        
    
    # show curve
    show_curve(losses, "train loss")
    show_curve(accs, "test accuracy")

In [5]:
def evaluate(model, val_loader, device):
    """
    model: CNN networks
    val_loader: a Dataloader object with validation data
    device: evaluate on cpu or gpu device
    return classification accuracy of the model on val dataset
    """
    # evaluate the model
    model.eval()
    # context-manager that disabled gradient computation
    with torch.no_grad():
        correct = 0
        total = 0
        
        for i, (images, targets) in enumerate(val_loader):
            # device: cpu or gpu
            images = images.to(device)
            targets = targets.to(device)
            
            
            outputs = model(images)
            
            # return the maximum value of each row of the input tensor in the 
            # given dimension dim, the second return vale is the index location
            # of each maxium value found(argmax)
            _, predicted = torch.max(outputs.data, dim=1)
            
            
            correct += (predicted == targets).sum().item()
            
            total += targets.size(0)
            
        accuracy = correct / total
        print('Accuracy on Test Set: {:.4f} %'.format(100 * accuracy))
        return accuracy

In [6]:
def save_model(model, save_path):
    # save model
    torch.save(model.state_dict(), save_path)

In [7]:
import matplotlib.pyplot as plt
def show_curve(ys, title):
    """
    plot curlve for Loss and Accuacy
    Args:
        ys: loss or acc list
        title: loss or accuracy
    """
    x = np.array(range(len(ys)))
    y = np.array(ys)
    plt.plot(x, y, c='b')
    plt.axis()
    plt.title('{} curve'.format(title))
    plt.xlabel('epoch')
    plt.ylabel('{}'.format(title))
    plt.show()

# dataLoader

In [8]:
# import torch
# import torch.nn as nn
# import torchvision
# import torchvision.transforms as transforms

# # mean and std of cifar10 in 3 channels 
# mean = (0.5,0.5,0.5)
# std = (0.5,0.5,0.5)

# # define transform operations of train dataset 
# data_transforms = {
#     'train' = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize(mean, std)]),

#     "val" = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize(mean, std)])
# }


# data_dir = 'I:\\1裂缝检测\\CrackForest-dataset\\trian\\crack\\'

# image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
#                                           data_transforms[x])	
#                   for x in ['train', 'val']}
# dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=100,
#                                              shuffle=True)
#               for x in ['train', 'val']}
# dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}


# # Data loader: provides single- or multi-process iterators over the dataset.
# train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
#                                            batch_size=256, 
#                                            shuffle=True)

# test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
#                                           batch_size=256, 
#                                           shuffle=False)

In [9]:
# Device configuration, cpu, cuda:0/1/2/3 available
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

optimizer = torch.optim.Adam(mycnn.parameters(), lr=lr)

# start training on cifar10 dataset
fit(cracknet, num_epochs, optimizer, device)

NameError: name 'torch' is not defined

# 数据准备

In [14]:
# coding=utf-8
import scipy.io as sio
import pandas as pd
import os
import numpy as np


path_dir = "./"
""" 将path_dir 文件夹下的.mat 文件转成二元数组array
因为该文件的数据较复杂：
struct groundTruth {
   Segmentation:320x480
   Boundaries:320x480
}  = loadmat(file_list)

下面的结果是：
dat[x][y] 就是 该320x480的某个值

或许需要将其生成图片

"""

nzero_list = []
zero_list = []

def mat2csv():

    curr_path = os.path.dirname(path_dir)
    mat_data_path = os.path.join(curr_path, "groundTruth")
#     csv_data_path = os.path.join(curr_path, "csv")
#     if not os.path.exists(csv_data_path):
#         os.makedirs(csv_data_path)
    if not os.path.exists(mat_data_path):
        os.makedirs(mat_data_path)
    file_list = os.listdir(mat_data_path)
    mat_list = [file_name for file_name in file_list if file_name.endswith(".mat")]
    print("find mat file : ", mat_list)
    

    for mat_file in mat_list:
        file_path = os.path.join(mat_data_path, mat_file)
        mat_data = sio.loadmat(file_path)
        for key in mat_data:
            if not str(key).startswith("__"):
                data = mat_data[key][:]
                print (mat_file)
                try:
                    dat = np.array(data[0]['Segmentation'][0])
                    temp = dat - 1
                    
                    nzero_list.append(np.nonzero(temp))
                    zero_list.append(np.argwhere(temp == 0))
                    
#                     print(dat[1][3])
#                     print(dat.shape)
                except ValueError as e:
                    print (e)
                    continue
if __name__ == "__main__":
    mat2csv()

find mat file :  ['071.mat', '038.mat', '093.mat', '087.mat', '026.mat', '085.mat', '013.mat', '039.mat', '012.mat', '105.mat', '106.mat', '080.mat', '024.mat', '048.mat', '007.mat', '077.mat', '023.mat', '084.mat', '016.mat', '075.mat', '059.mat', '076.mat', '103.mat', '086.mat', '035.mat', '031.mat', '058.mat', '027.mat', '010.mat', '047.mat', '065.mat', '046.mat', '101.mat', '022.mat', '074.mat', '056.mat', '112.mat', '115.mat', '082.mat', '067.mat', '060.mat', '095.mat', '055.mat', '102.mat', '104.mat', '062.mat', '079.mat', '005.mat', '018.mat', '044.mat', '100.mat', '042.mat', '030.mat', '094.mat', '072.mat', '092.mat', '032.mat', '096.mat', '091.mat', '109.mat', '111.mat', '066.mat', '019.mat', '006.mat', '081.mat', '028.mat', '001.mat', '033.mat', '108.mat', '052.mat', '021.mat', '041.mat', '017.mat', '011.mat', '043.mat', '051.mat', '097.mat', '064.mat', '089.mat', '088.mat', '037.mat', '073.mat', '090.mat', '116.mat', '002.mat', '118.mat', '099.mat', '008.mat', '061.mat', '00

In [15]:
# print (point.shape())
import numpy as np

temp = np.array(nzero_list)
print(temp.shape)

(118, 2)


In [16]:
print(nzero_list[1])
x,y = nzero_list[:][0],nzero_list[:][1]
# x = np.array(nzero_list[:][0])
# y = np.array(nzero_list[:][1])
# print(x[1][1],y[1][1])

(array([ 88,  89,  89, ..., 170, 170, 170]), array([  0,   0,   1, ..., 473, 474, 475]))


In [17]:
# r = np.array(temp[2][0])
# t = np.array(temp[2][1])

# 读取图片分割并写入本地


In [18]:
from PIL import Image
import numpy as np
def test():
    #读取一张图片，输出矩阵
    imgAddress = "./image/"
    im = Image.open(imgAddress+"001.jpg")
    print(im.size)
    arr = np.array(im)
    print(arr.shape)
test()

(480, 320)
(320, 480, 3)


In [19]:
# -*- coding:utf-8 -*-

from PIL import Image
def cut():
    # 图片地址
    imgAddress = "./image/"
    imgSave = "./crack/"
    for i in range(1):
        i = i + 1
        x_list = np.array(temp[i-1][1])
        y_list = np.array(temp[i-1][0])
#         str = sprintf("%3d",i);
#         str = str + ".jpg"
        str = "{:0>3d}.jpg".format(i);
        
        print(str)
        im = Image.open(imgAddress + str )
        for j in range(len(x_list)):
            x1 = x_list[j]-13
            y1 = y_list[j]-13
            x2 = x_list[j]+14
            y2 = y_list[j]+14
            im2 = im.crop((x1,y1,x2,y2))
#             print(imgSave+"{:0>3d}{:0>5d}.jpg".format(i,j))
            im2.save(imgSave+"{:0>3d}{:0>5d}.jpg".format(i,j))
            
            
        
        
        

In [20]:
x_list = np.array(temp[0][0])
y_list = np.array(temp[0][1])
# print(x_list[2376],y_list[2376])
print(x_list.size,y_list.size)

4018 4018


# 数据集
先别纠结了，直接将图片生成到本地，然后再导入处理
下面是opencv剪切图片并保存到本地

In [21]:
cut()

001.jpg


# 剪切负样本

In [35]:
# coding=utf-8
import scipy.io as sio
import pandas as pd
import os
import numpy as np


path_dir = "./"
""" 将path_dir 文件夹下的.mat 文件转成二元数组array
因为该文件的数据较复杂：
struct groundTruth {
   Segmentation:320x480
   Boundaries:320x480
}  = loadmat(file_list)

下面的结果是：
dat[x][y] 就是 该320x480的某个值

或许需要将其生成图片

"""

nzero_list = []
zero_list = []

def mat2csv():

    curr_path = os.path.dirname(path_dir)
    mat_data_path = os.path.join(curr_path, "groundTruth")
#     csv_data_path = os.path.join(curr_path, "csv")
#     if not os.path.exists(csv_data_path):
#         os.makedirs(csv_data_path)
    if not os.path.exists(mat_data_path):
        os.makedirs(mat_data_path)
    file_list = os.listdir(mat_data_path)
    mat_list = [file_name for file_name in file_list if file_name.endswith(".mat")]
    print("find mat file : ", mat_list)
    

    for mat_file in mat_list:
        file_path = os.path.join(mat_data_path, mat_file)
        mat_data = sio.loadmat(file_path)
        for key in mat_data:
            if not str(key).startswith("__"):
                data = mat_data[key][:]
                print (mat_file)
                try:
                    dat = np.array(data[0]['Segmentation'][0])
                    temp = dat - 1
                    nzero_list.append(np.argwhere(temp != 0))
                    zero_list.append(np.argwhere(temp == 0))
                    
#                     print(dat[1][3])
#                     print(dat.shape)
                except ValueError as e:
                    print (e)
                    continue
if __name__ == "__main__":
    mat2csv()

find mat file :  ['071.mat', '038.mat', '093.mat', '087.mat', '026.mat', '085.mat', '013.mat', '039.mat', '012.mat', '105.mat', '106.mat', '080.mat', '024.mat', '048.mat', '007.mat', '077.mat', '023.mat', '084.mat', '016.mat', '075.mat', '059.mat', '076.mat', '103.mat', '086.mat', '035.mat', '031.mat', '058.mat', '027.mat', '010.mat', '047.mat', '065.mat', '046.mat', '101.mat', '022.mat', '074.mat', '056.mat', '112.mat', '115.mat', '082.mat', '067.mat', '060.mat', '095.mat', '055.mat', '102.mat', '104.mat', '062.mat', '079.mat', '005.mat', '018.mat', '044.mat', '100.mat', '042.mat', '030.mat', '094.mat', '072.mat', '092.mat', '032.mat', '096.mat', '091.mat', '109.mat', '111.mat', '066.mat', '019.mat', '006.mat', '081.mat', '028.mat', '001.mat', '033.mat', '108.mat', '052.mat', '021.mat', '041.mat', '017.mat', '011.mat', '043.mat', '051.mat', '097.mat', '064.mat', '089.mat', '088.mat', '037.mat', '073.mat', '090.mat', '116.mat', '002.mat', '118.mat', '099.mat', '008.mat', '061.mat', '00

In [49]:
temp = np.array(zero_list)
print(temp.shape)
print(temp.size)

(118,)
118


In [50]:
t = np.array(nzero_list)


236


In [52]:
t = np.array(t[0])
print(t.size)

4018


In [37]:
tt = np.array(temp[0])

In [46]:
temp = np.array(zero_list)
print(temp.shape)

from PIL import Image
def cut_negative():
    # 图片地址
    imgAddress = "./image/"
    imgSave = "./no_crack/"
    for i in range(1):
        i = i + 1
        index_list = np.array(temp[i-1])
#         str = sprintf("%3d",i);
#         str = str + ".jpg"
        str = "{:0>3d}.jpg".format(i);
        print(str)
        im = Image.open(imgAddress + str)
        for j in range(len(index_list)//27):
            j = j * 20
            if((index_list[j][1] > 27 and index_list[j][1] < 453)and (index_list[j][0] > 27 and index_list[j][0] < 293)):
                x1 = index_list[j][1]-13
                y1 = index_list[j][0]-13
                x2 = index_list[j][1]+14
                y2 = index_list[j][0]+14
                im2 = im.crop((x1,y1,x2,y2))
    #             print(imgSave+"{:0>3d}{:0>5d}.jpg".format(i,j))
                im2.save(imgSave+"{:0>3d}{:0>7d}.jpg".format(i,j))



(118,)


In [47]:
cut_negative()

001.jpg


# 剪切正样本


In [106]:
# coding=utf-8
import scipy.io as sio
import pandas as pd
import os
import numpy as np


path_dir = "./"
""" 将path_dir 文件夹下的.mat 文件转成二元数组array
因为该文件的数据较复杂：
struct groundTruth {
   Segmentation:320x480
   Boundaries:320x480
}  = loadmat(file_list)

下面的结果是：
dat[x][y] 就是 该320x480的某个值

或许需要将其生成图片

"""

nzero_list = []
zero_list = []

def mat2csv():

    curr_path = os.path.dirname(path_dir)
    mat_data_path = os.path.join(curr_path, "groundTruth")
    if not os.path.exists(mat_data_path):
        os.makedirs(mat_data_path)
    file_list = os.listdir(mat_data_path)
    mat_list = [file_name for file_name in file_list if file_name.endswith(".mat")]
    print("find mat file : ", mat_list)
    

    for mat_file in mat_list:
        file_path = os.path.join(mat_data_path, mat_file)
        mat_data = sio.loadmat(file_path)
        for key in mat_data:
            if not str(key).startswith("__"):
                data = mat_data[key][:]
                print (mat_file)
                try:
                    dat = np.array(data[0]['Segmentation'][0])
                    temp = dat - 1
                    nzero_list.append(np.argwhere(temp != 0))
                except ValueError as e:
                    print (e)
                    continue
if __name__ == "__main__":
    mat2csv()

find mat file :  ['071.mat', '038.mat', '093.mat', '087.mat', '026.mat', '085.mat', '013.mat', '039.mat', '012.mat', '105.mat', '106.mat', '080.mat', '024.mat', '048.mat', '007.mat', '077.mat', '023.mat', '084.mat', '016.mat', '075.mat', '059.mat', '076.mat', '103.mat', '086.mat', '035.mat', '031.mat', '058.mat', '027.mat', '010.mat', '047.mat', '065.mat', '046.mat', '101.mat', '022.mat', '074.mat', '056.mat', '112.mat', '115.mat', '082.mat', '067.mat', '060.mat', '095.mat', '055.mat', '102.mat', '104.mat', '062.mat', '079.mat', '005.mat', '018.mat', '044.mat', '100.mat', '042.mat', '030.mat', '094.mat', '072.mat', '092.mat', '032.mat', '096.mat', '091.mat', '109.mat', '111.mat', '066.mat', '019.mat', '006.mat', '081.mat', '028.mat', '001.mat', '033.mat', '108.mat', '052.mat', '021.mat', '041.mat', '017.mat', '011.mat', '043.mat', '051.mat', '097.mat', '064.mat', '089.mat', '088.mat', '037.mat', '073.mat', '090.mat', '116.mat', '002.mat', '118.mat', '099.mat', '008.mat', '061.mat', '00

In [107]:
temp = np.array(nzero_list)
print(temp.shape)

(118,)


In [110]:
tt = np.array(temp[13])
print(tt.size)

3678


In [116]:
temp = np.array(nzero_list)
print(temp.shape)

from PIL import Image
def cut_positive():
    # 图片地址
    imgAddress = "./image/"
    imgSave = "./crack/"
    for i in range(1):
        i = i + 10
        index_list = np.array(temp[i-1])
#         str = sprintf("%3d",i);
#         str = str + ".jpg"
        str = "{:0>3d}.jpg".format(i);
        print(str)
        im = Image.open(imgAddress + str)
        for j in range(len(index_list)):
            x1 = index_list[j][1]-13
            y1 = index_list[j][0]-13
            x2 = index_list[j][1]+14
            y2 = index_list[j][0]+14
            im2 = im.crop((x1,y1,x2,y2))
    #             print(imgSave+"{:0>3d}{:0>5d}.jpg".format(i,j))
            im2.save(imgSave+"{:0>3d}{:0>7d}.jpg".format(i,j))



(118,)


In [117]:
cut_positive()

010.jpg


In [115]:
import os 
import shutil as sh 

positivePath = "./crack/"
negativePath = "./no_crack/"

train_po_path = "./data/train/crack/"
train_ne_path = "./data/train/no_crack/"

val_po_path = "./data/val/crack/"
val_ne_path = "./data/val/no_crack/"


po_file_list = os.listdir(positivePath)
ne_file_list = os.listdir(negativePath)
# print(po_file_list)

for j in range(len(po_file_list)):
    if(j < 100000):
        sh.move(positivePath+po_file_list[j],train_po_path+po_file_list[j])
    else:
        sh.move(positivePath+po_file_list[j],val_po_path+po_file_list[j])
    if(j % 10000 == 0):
        print(j)
#     if(j > 2000):
#         break;

for j in range(len(ne_file_list)):
    if(j < 200000):
        sh.move(negativePath+ne_file_list[j],train_ne_path+ne_file_list[j])
    else:
        sh.move(negativePath+ne_file_list[j],val_ne_path+ne_file_list[j])
#     if(j > 4000):
#         break
    if(j % 1000 == 0):
        print(j)

    

0
0
1000
2000
3000
4000


In [15]:
import math
import torch.nn as nn

class Crack(nn.Module):
    def __init__(self, Crack_cfg):
        super(Crack, self).__init__()
        self.features = self._make_layers(Crack_cfg)
        # linear layer
#         self.classifier = nn.Linear(512, 10)
        self.linear1 = nn.Linear(32*6*6,64)
#         self.linear1 = nn.Sequential([nn.Linear((32*6*6,64),
#                                  nn.Sigmoid())])
        self.linear2 = nn.Linear(64,64)
#         self.linear2 = nn.Sequential([nn.Linear((64,64),
#                                  nn.Sigmoid())])
        self.linear3 = nn.Linear(64,2)
#         self.linear3 = nn.Sequential([nn.Linear((64,25),
#                                  nn.Sigmoid())])

    def forward(self, x):
        out = self.features(x)
#         print(out.size())
        out = out.view(out.size(0), -1)
        out = self.linear1(out)
        out = self.linear2(out)
        out = self.linear3(out)
        return out

    def _make_layers(self, cfg):
        """
        cfg: a list define layers this layer contains
            'M': MaxPool, number: Conv2d(out_channels=number) -> BN -> ReLU
        """
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
            
#         layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)

In [19]:
Crack_cfg = {
    'Crack11':[16,16,'M',32,32,'M']
}

model_t = Crack(Crack_cfg['Crack11']);
model_t


Crack(
  (features): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU(inplace)
    (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
    (5): ReLU(inplace)
    (6): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
    (7): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
    (9): ReLU(inplace)
    (10): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
    (12): ReLU(inplace)
    (13): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=1152, out_features=64, bias=True)
    (1): ReLU(inplace)
    (2): Dropout(p=0.5)
   

In [21]:

# License: BSD
# Author: Sasank Chilamkurthy

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train' : transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))]),
    "val" : transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])
}

data_dir = './data/'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])	
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=256,
                                             shuffle=True)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

# imshow(out, title=[class_names[x] for x in classes])

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    # 这里可能是使用了模型库里面的模型？！
    best_model_wts = copy.deepcopy(model.state_dict())
    # best_acc = 0.0
    best_pr = 0.0
    best_re = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            L0 = 0
            P0 = 0
            P_neq = 0
            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)

                #L0 = TP + FN,即原本的裂缝数据，包括真裂缝和假非裂缝
                L0 += torch.sum(labels == 0)
                
                #P0 = TP+FP ,即预测为裂缝的数据，包括真裂缝和假裂缝
                P0 += torch.sum(preds == 0)
                #P1 = FN+TN,即预测为非裂缝的数据，包括真非裂缝和假非裂缝
                P_neq += torch.sum(preds != labels)
            
#             print(L0.numpy().size,P0.numpy().size,P1.numpy().size)
            t = L0+P0-P_neq
            print(t)
            print(2*P0)
            print(2*L0)
            #经计算，TP = (L0+P0-P1)/2
            #则PR = (L0+PO-P1)/(2*P0)
            PRECISE = t.float()/(2*P0)
            #RE = (L0+P0-P1)/(2*L0)，即原本的裂缝数据，有多少被检测出来了
            RECALL = t.float()/(2*L0)

            epoch_loss = running_loss / dataset_sizes[phase]
            # epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} PRE: {:.4f} REC: {:.4f}'.format(
                phase, epoch_loss, PRECISE,RECALL))
            # deep copy the model
            if phase == 'val' and (PRECISE >= best_pr or RECALL >= best_re):
                best_re = RECALL
                best_pr = PRECISE
                best_model_wts = copy.deepcopy(model.state_dict())
        print()
  
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    # fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                # ax = plt.subplot(num_images//2, 2, images_so_far)
                # ax.axis('off')
                # ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                # imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

Crack_cfg = {
    'Crack11':[16,16,'M',32,32,'M']
}

model_ft = Crack(Crack_cfg['Crack11']);

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs 退火
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=20)
model_ft = model_ft.to('cpu')
torch.save(model_ft.cpu().state_dict(),"./crack.pt")
# # visualize_model(model_ft)
# model_conv = torchvision.models.resnet18(pretrained=True)
# for param in model_conv.parameters():
#     param.requires_grad = False

# # Parameters of newly constructed modules have requires_grad=True by default
# num_ftrs = model_conv.fc.in_features
# model_conv.fc = nn.Linear(num_ftrs, 2)

# model_conv = model_conv.to(device)

# criterion = nn.CrossEntropyLoss()

# # Observe that only parameters of final layer are being optimized as
# # opposed to before.
# optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# # Decay LR by a factor of 0.1 every 7 epochs
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)
# model_conv = train_model(model_conv, criterion, optimizer_conv,
#                          exp_lr_scheduler, num_epochs=25)
# model_conv = model_conv.to('cpu')
# torch.save(model_conv,"./model1.pt")
# # visualize_model(model_conv)

# # plt.ioff()
# # plt.show()

AttributeError: module 'torch' has no attribute 'device'

In [23]:
import torch
device = torch.device("gup")

AttributeError: module 'torch' has no attribute 'device'

In [12]:
tt = np.array(([1,0,0],[0,1,0],[1,0,1]))
ff = np.array(([1,1,0],[0,0,0],[1,0,1]))

In [22]:
t = np.array(tt == ff)


In [23]:
t


array([[ True, False,  True],
       [ True, False,  True],
       [ True,  True,  True]])

In [24]:
f = np.array(tt == 1)
f

array([[ True, False, False],
       [False,  True, False],
       [ True, False,  True]])

In [25]:
g = np.arange(f == t)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()